In [1]:
import matplotlib.pyplot as plt
import numpy as np
import argparse
import imutils
import time
import cv2
import os

In [2]:
from google.colab import files
uploaded = files.upload()


Saving yolo-coco.zip to yolo-coco.zip


In [3]:
!unzip yolo-coco.zip

Archive:  yolo-coco.zip
   creating: yolo-coco/
  inflating: yolo-coco/coco.names    
  inflating: yolo-coco/yolov3.cfg    
  inflating: yolo-coco/yolov3.weights  


In [4]:
#  Upload video
from google.colab import files
uploaded = files.upload()


Saving overpass.mp4 to overpass.mp4


In [39]:
# Using YOLO for object detection.Defining input video, output video name, 
# confidence :Minimum probability to filter weak detections
# threshold :non-maxima suppression threshold
args = {
    "input": "/content/overpass.mp4",
    "output": "/content/overpass.avi",
    "yolo": "yolo-coco",
    "confidence": 0.5,
    "threshold": 0.3
}

In [40]:
# load the COCO class labels YOLO model was trained on
labelsPath = os.path.sep.join([args["yolo"], "coco.names"])
LABELS = open(labelsPath).read().strip().split("\n")

# initialize a list of colors to represent each possible class label
np.random.seed(42)
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3),
	dtype="uint8")

In [41]:
# derive the paths to the YOLO weights and model configuration
weightsPath = os.path.sep.join([args["yolo"], "yolov3.weights"])
configPath = os.path.sep.join([args["yolo"], "yolov3.cfg"])

In [42]:
# load YOLO object detector trained on COCO dataset (80 classes)
# and determine only the *output* layer names that are needed from YOLO
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)
ln = net.getLayerNames()
print(ln)
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]
print(ln)

['conv_0', 'bn_0', 'relu_0', 'conv_1', 'bn_1', 'relu_1', 'conv_2', 'bn_2', 'relu_2', 'conv_3', 'bn_3', 'relu_3', 'shortcut_4', 'conv_5', 'bn_5', 'relu_5', 'conv_6', 'bn_6', 'relu_6', 'conv_7', 'bn_7', 'relu_7', 'shortcut_8', 'conv_9', 'bn_9', 'relu_9', 'conv_10', 'bn_10', 'relu_10', 'shortcut_11', 'conv_12', 'bn_12', 'relu_12', 'conv_13', 'bn_13', 'relu_13', 'conv_14', 'bn_14', 'relu_14', 'shortcut_15', 'conv_16', 'bn_16', 'relu_16', 'conv_17', 'bn_17', 'relu_17', 'shortcut_18', 'conv_19', 'bn_19', 'relu_19', 'conv_20', 'bn_20', 'relu_20', 'shortcut_21', 'conv_22', 'bn_22', 'relu_22', 'conv_23', 'bn_23', 'relu_23', 'shortcut_24', 'conv_25', 'bn_25', 'relu_25', 'conv_26', 'bn_26', 'relu_26', 'shortcut_27', 'conv_28', 'bn_28', 'relu_28', 'conv_29', 'bn_29', 'relu_29', 'shortcut_30', 'conv_31', 'bn_31', 'relu_31', 'conv_32', 'bn_32', 'relu_32', 'shortcut_33', 'conv_34', 'bn_34', 'relu_34', 'conv_35', 'bn_35', 'relu_35', 'shortcut_36', 'conv_37', 'bn_37', 'relu_37', 'conv_38', 'bn_38', 're

In Yolo detections are made are at the 82nd layer, 94th layer, and 106th layer.
For the first detection, the first 81 layers are downsampled such that the 81st layer has a stride of 32 resulting in first feature map of size 13x13 and the first detection is made with a 1x1 kernel, leading to  detection 3D tensor of size 13x13x255.
For the second detection, the 79th layer onwards is subjected to convolutional layers before upsampling to dimensions 26x26. This feature map is then depth concatenated with the feature map from layer 61 to form a new feature map which is further fused with the 61st layer with the help of 1x1 convolution layers. The second detection layer is at the 94th layer with a 3D tensor of size 26x26x255.
For the final(third) detection layer, the same process is followed as that of the second detection where the feature map of the 91st layer is subjected to convolution layers before being depth concatenated and fused with a feature map from 36th layer. The final detection is made at the 106th layer with a feature map of size 52x52x255.

Reference : https://towardsdatascience.com/digging-deep-into-yolo-v3-a-hands-on-guide-part-1-78681f2c7e29



In [43]:
# initialize the video stream, pointer to output video file, and
# frame dimensions
vs = cv2.VideoCapture(args["input"])
print(vs)
writer = None
(W, H) = (None, None)

<VideoCapture 0x7fc10af59810>


In [44]:
cv2.__version__

'4.1.2'

In [45]:
total = int(vs.get(cv2.CAP_PROP_FRAME_COUNT)) # As cv2 version is 4.x this function is used
print(total) # Number of frames in video file

812


In [46]:


while True: # loop over frames from the video file stream
  (grabbed,frame) = vs.read() # reading next frames in files
  if not grabbed: # If no frame exists then exit
    break
  elif W is None or H is None: # If frame dimensions are empty, grab them
    (H, W) = frame.shape[:2]

  blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416),swapRB=True, crop=False) #blobFromImage performs Mean Subtraction, Scaling and channel swapping
  net.setInput(blob)
  start = time.time()
  layerOutputs = net.forward(ln)
  end = time.time()
  # Initialize lists of bounding boxes, confidence scores and class labels
  boxes = [] 
  confidences = []
  classIDs = []
  for output in layerOutputs: #Loop over output layers
    for detection in output: # Loop over detection
      scores = detection[5:] # Get class label and confidence of detection
      classID = np.argmax(scores)
      confidence = scores[classID]
      if confidence > args["confidence"]:# if probability is greater than the minimum probability
        box = detection[0:4] * np.array([W, H, W, H]) #scale the bounding box coordinates back relative to the size of the image
				#YOLO actually returns the center (x, y)-coordinates of the bounding box followed by the boxes' width and height
        (centerX, centerY, width, height) = box.astype("int")
        #use the center (x, y)-coordinates to derive the right and left corner of the bounding box
        x = int(centerX - (width / 2))
        y = int(centerY - (height / 2))
        # Update lists
        boxes.append([x, y, int(width), int(height)])
        confidences.append(float(confidence))
        classIDs.append(classID)
  # use non maxima supression to supress weak, overlapping boxes
  idxs = cv2.dnn.NMSBoxes(boxes, confidences, args["confidence"],args["threshold"])
  if len(idxs) > 0:
    for i in idxs.flatten():# loop over the indexes we are keeping
      (x, y) = (boxes[i][0], boxes[i][1]) # extract the bounding box coordinates
      (w, h) = (boxes[i][2], boxes[i][3])
      # draw a bounding box rectangle and label on the frame
      color = [int(c) for c in COLORS[classIDs[i]]]
      cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
      text = "{}: {:.4f}".format(LABELS[classIDs[i]],confidences[i])
      cv2.putText(frame, text, (x, y - 5),cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
  if writer is None:
    fourcc = cv2.VideoWriter_fourcc(*"MJPG") # initialize video writer
    writer = cv2.VideoWriter(args["output"], fourcc, 30,(frame.shape[1], frame.shape[0]), True)
    if total > 0:
      elap = (end - start)
      print("[INFO] single frame took {:.4f} seconds".format(elap))
      print("[INFO] estimated total time to finish: {:.4f}".format(elap * total))
  writer.write(frame) # write the output frame to disk

# release the file pointers
print("[INFO] cleaning up...")
writer.release()
vs.release()
  


[INFO] single frame took 2.5620 seconds
[INFO] estimated total time to finish: 2080.3560
[INFO] cleaning up...


blobFromImage explanation:
Mean subtraction is used to help combat illumination changes in the input images in dataset. 
Scaling factor, \sigma, adds in a normalization
spatial size that the Convolutional Neural Network expects is given as (416,416)
